In [1]:
import json
import os
import pandas as pd

In [2]:
data_dir="C:/Users/tsb7592/Box/Baby Toolbox Operations/Domain Teams/Language/Tobii Pilot/LWL & EF Tobii Data Export"

In [3]:
os.chdir(data_dir)

In [66]:
tdf= pd.read_csv('Baby Toolbox_Data_Export_06_02_2021.tsv', sep='\t') 

C:\Users\tsb7592\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (28,34,36) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [67]:
tdf.shape

(305387, 380)

In [68]:
keep = [col for col in tdf.columns if 'AOI' in col]
keep = [col for col in keep if 'Calib' not in col]

In [69]:
k24 = keep[4:76]

In [70]:
print(k24)

['AOI[LeftStim_Shoulder]Hit', 'AOI[RightStim_Hose]Hit', 'AOI[WholeScreen]Hit.4', 'AOI[LeftStim_Ladder]Hit', 'AOI[RightStim_Melon]Hit', 'AOI[WholeScreen]Hit.5', 'AOI[LeftStim_Rooster]Hit', 'AOI[RightStim_Playground]Hit', 'AOI[WholeScreen]Hit.6', 'AOI[LeftStim_Bike]Hit', 'AOI[RightStim_Sneaker]Hit', 'AOI[WholeScreen]Hit.7', 'AOI[LeftStim]Hit', 'AOI[RightStim]Hit', 'AOI[WholeScreen]Hit.8', 'AOI[LeftStim]Hit.1', 'AOI[RightStim]Hit.1', 'AOI[WholeScreen]Hit.9', 'AOI[LeftStim]Hit.2', 'AOI[RightStim]Hit.2', 'AOI[WholeScreen]Hit.10', 'AOI[LeftStim]Hit.3', 'AOI[RightStim]Hit.3', 'AOI[WholeScreen]Hit.11', 'AOI[LeftStim]Hit.4', 'AOI[RightStim]Hit.4', 'AOI[WholeScreen]Hit.12', 'AOI[LeftStim]Hit.5', 'AOI[RightStim]Hit.5', 'AOI[WholeScreen]Hit.13', 'AOI[LeftStim]Hit.6', 'AOI[RightStim]Hit.6', 'AOI[WholeScreen]Hit.14', 'AOI[LeftStim]Hit.7', 'AOI[RightStim]Hit.7', 'AOI[WholeScreen]Hit.15', 'AOI[LeftStim]Hit.8', 'AOI[RightStim]Hit.8', 'AOI[WholeScreen]Hit.16', 'AOI[LeftStim]Hit.9', 'AOI[RightStim]Hit.9'

In [71]:
tdf=tdf[k24]

In [72]:
tdata = round(tdf.mean(),4).to_dict()

In [73]:
left =[]
right=[]
away =[]
onscreen = []
i = 0
for val in tdata:
    if i%3 == 0:
        rv = tdata[val]
        left.append(tdata[val])
    elif i%3 == 1:
        rv += tdata[val]
        right.append(tdata[val])
    else:
        rv = 1 - rv
        away.append(rv)
        onscreen.append(tdata[val])
    i+=1

In [74]:
k24[3:6]

['AOI[LeftStim_Ladder]Hit', 'AOI[RightStim_Melon]Hit', 'AOI[WholeScreen]Hit.5']

# json file

In [4]:
data_dir="C:/Users/tsb7592/Box/Baby Toolbox Operations/Domain Teams/Language/Tobii Pilot/Tobii Verbose Files/Claire"

In [5]:
os.chdir(data_dir)

In [ ]:
newfile ='949BCA6A-D3D6-426C-9A94-E806F84E4E72@845BDD10-B9AF-4592-AA4E-0CCE76A0E8CD@00025@2021-05-11-22.21.08.674@ExecutiveFunction@Claire@VerboseGazeData.json'

In [ ]:
#df=pd.read_json(newfile)

In [6]:
df=pd.read_json('949BCA6A-D3D6-426C-9A94-E806F84E4E72@845BDD10-B9AF-4592-AA4E-0CCE76A0E8CD@00024@2021-05-11-22.11.48.356@LookListening@Claire@VerboseGazeData.json')

In [ ]:
cols=['itemID','eventName','gazeLocationName', 'gazeLocationOnScreen']

In [ ]:
df=df[cols]

In [ ]:
df.eventName.unique()

In [ ]:
len(df.gazeLocationOnScreen.value_counts())

In [ ]:
#Finding the index for presentLiveItem and completeLiveItem to subset the pieces we need for analysis
begin =list(df[df['eventName'].str.contains('presentedLiveItem')].index)

In [ ]:
end = list(df[df['eventName'].str.contains('completedLiveItem')].index)

In [ ]:
expected = pd.DataFrame(
    {'Left': left,
     'Right': right,
     'Away': away,'Onscreen':onscreen
    })
expected

In [ ]:
len(end)

# Fisher Exact Test 

In [ ]:
#tsv file
exp = pd.DataFrame(columns=('left', 'right','onScreenAway','offScreenAway'))
for i in range(24):
    lcol,rcol,wcol = k24[3*i:3*i+3]
    
    subset = tdf[tdf[wcol].notnull()]
    
    a=subset[lcol].sum()
    b=subset[rcol].sum()
    d=subset[(subset[lcol]==0) & (subset[rcol]==0) &(subset[wcol]==1)].shape[0]
    e=subset[(subset[lcol]==0) & (subset[rcol]==0) &(subset[wcol]==0)].shape[0]

    exp.loc[i] = [a, b ,d, e]

In [ ]:
#json file
obs = pd.DataFrame(columns=('L', 'R','onScreenAway','offScreenAway'))
items =[]
for i in range(len(end)):
    sub = df.iloc[begin[i]+1:end[i]]
    items.extend(sub.itemID.dropna().unique())
    a, b, c = sub.gazeLocationName.value_counts()[['left','right','away']].round(4) 
    if len(sub[sub['gazeLocationName'] == 'away'].gazeLocationOnScreen.value_counts()) == 2:
        d,e = sub[sub['gazeLocationName'] == 'away'].gazeLocationOnScreen.value_counts()
    else:
        d = c
        e = 0
    obs.loc[i] = [a, b ,d, e]

In [ ]:
exp

In [ ]:
obs

In [ ]:
exp.to_csv('C:/Users/tsb7592/Box/Baby Toolbox Operations/Analysis/new_tsv_count.csv', index=False)

In [ ]:
obs.to_csv('C:/Users/tsb7592/Box/Baby Toolbox Operations/Analysis/json_count.csv', index=False)

In [ ]:
l1 =obs.to_numpy()
l2= exp.to_numpy()

In [ ]:
items

In [ ]:
import scipy.stats as stats
from scipy.stats import chi2_contingency
from scipy.stats import chi2

for i in range(24):
    data = [l1[i], l2[2]]
    
    stat, p, dof, expected = chi2_contingency(data)
    print(items[i])
    
    # interpret test-statistic
    prob = 0.95
    critical = chi2.ppf(prob, dof)
    print('probability=%.3f, critical=%.3f, stat=%.3f' % (prob, critical, stat))
    if abs(stat) >= critical:
        print('Dependent (reject H0)')
    else:
        print('Independent (fail to reject H0)')

    # interpret p-value
    alpha = 1.0 - prob
    print('significance=%.3f, p=%.3f' % (alpha, p))
    if p <= alpha:
        print('Dependent (reject H0)')
    else:
        print('Independent (fail to reject H0)')
    print('')

In [ ]:
from scipy.stats import chisquare
chisq = pd.DataFrame(columns=('items', 'stat','p'))
for i in range(24):
    ob = obs.iloc[i]
    ex = expected.iloc[i]
    stat, pv = chisquare(f_obs=ob, f_exp=ex)
    chisq.loc[i] = [items[i], round(stat,3) ,round(pv,3)]


In [ ]:
chisq.to_csv('C:/Users/TEMP/Box/Baby Toolbox Operations/Analysis/chisquare.csv', index=False)

In [ ]:
startRows =list(df[df['eventName'].str.contains('presentedCalibrationItem')].index)
endRows =list(df[df['eventName'].str.contains('completedCalibrationItem')].index)

# Rewrite Lookalgorithm

In [1]:
import json
import os
import pandas as pd

In [2]:
data_dir="C:/Users/tsb7592/Box/Baby Toolbox Operations/Domain Teams/Language/Tobii Pilot/Tobii Verbose Files/Claire"

In [3]:
os.chdir(data_dir)

In [4]:
df=pd.read_json('949BCA6A-D3D6-426C-9A94-E806F84E4E72@845BDD10-B9AF-4592-AA4E-0CCE76A0E8CD@00024@2021-05-11-22.11.48.356@LookListening@Claire@VerboseGazeData.json')

In [5]:
import math
import numpy as np

def lookAlgorithm(df, usevals, searchTime=1.0167, margX=10, margY=15, alignX=30, alignY=30):
    '''
    Input: 
        df: the json file read to a pandas dataframe
        usevals: a string of the column name
    Output:
         results in a pandas dataframe
    '''
    
    if 'lookAtPoint' in usevals:
        useX = "lookAtPointX"
        useY = "lookAtPointY"
    elif 'arHitTestLookAtPointPosition' in usevals:
        useX = "arHitTestLookAtPointPositionX"
        useY = "arHitTestLookAtPointPositionY"
    
    startRows =list(df[df['eventName'].str.contains('presentedCalibrationItem')].index)
    endRows =list(df[df['eventName'].str.contains('completedCalibrationItem')].index)
    
    if len(startRows) != len(endRows):
        print("Early-terminated Calibration")
        return
    
    RMs=[]
    for i in range(len(startRows)):
        rollingMeans = pd.DataFrame(columns=('Cali','calibFocal', 'rowIndex','rollingX','rollingY','props')) 
        sub = df.iloc[startRows[i]:endRows[i]+1]
        firstCheck_time = df.iloc[startRows[i]]['elapsedTime'] + searchTime
        firstCheck_row = sub[sub['elapsedTime'] <= firstCheck_time].index.max()
       
        for j in range(firstCheck_row, endRows[i]+1):
            rollingMeans.loc[j] = [i,'tmp', j, -9999,-9999, -9999]
            
            useData =sub[(sub['elapsedTime'] >= df.iloc[j]['elapsedTime'] - searchTime) & (sub['elapsedTime'] <= df.iloc[j]['elapsedTime'])&(sub['elapsedTime'] >= df.iloc[startRows[i]]['elapsedTime'])]
            useData = useData[~useData.eventName.str.contains('complet')]
                              
            rollingMeans.loc[j]['calibFocal']=useData['calibrationFocalPoint'].unique()[0]
            rollingMeans.loc[j]['rollingX']=round(useData[useX].mean(),2)
            rollingMeans.loc[j]['rollingY']=round(useData[useY].mean(),2)
            rollingMeans.loc[j]['props']=round(100*(useData[(useData[useX] >= useData[useX].mean()- margX) & (useData[useX] <= useData[useX].mean()+ margX) & (useData[useY] >= useData[useY].mean()- margY) & (useData[useY] <= useData[useY].mean()+ margY)]).shape[0]/useData[useData[useY].notnull()].shape[0],2)
          
        RMs.append(rollingMeans)
         
    RMs = pd.concat(RMs, ignore_index=True)
    
    use=[]
    for i in range(4):
        ur = RMs[(RMs['props'] >= 80) & (RMs['Cali']==i)].head(1)
        use.append(ur)
    useRows = pd.concat(use)
    useRows = useRows.reset_index()
    
    #if useRows.shape[0]>4:
    useRows['calibrationSet'] =(useRows.index + 1)/4
    useRows['calibrationSet'] =useRows['calibrationSet'].apply(np.ceil)
        
    tr = useRows[useRows['calibFocal']=='topRight']['rollingX'].values[0]
    br = useRows[useRows['calibFocal']=='bottomRight']['rollingX'].values[0]
    tl = useRows[useRows['calibFocal']=='topLeft']['rollingX'].values[0]
    bl = useRows[useRows['calibFocal']=='bottomLeft']['rollingX'].values[0]    
    
    #save the 4 screenSide numbers to a dictionary 
    screenSide={}
    if abs(tr- br) <= alignX:
        screenSide['rightSide'] = max(tr, br)
    else:
        screenSide['rightSide'] = np.NaN
    
    if abs(tl- bl) <= alignX:
        screenSide['leftSide']= min(tl, bl)
    else:
        screenSide['leftSide']= np.NaN
    
    tr = useRows[useRows['calibFocal']=='topRight']['rollingY'].values[0]
    br = useRows[useRows['calibFocal']=='bottomRight']['rollingY'].values[0]
    tl = useRows[useRows['calibFocal']=='topLeft']['rollingY'].values[0]
    bl = useRows[useRows['calibFocal']=='bottomLeft']['rollingY'].values[0]
    
    if abs(tr- tl) <= alignY:
        screenSide['topSide'] = round((tr+tl)/2,2)
    else:
        screenSide['topSide'] = np.NaN
        
    if abs(br- bl) <= alignY:
        screenSide['bottomSide']= round((br+bl)/2,2)
    else:
        screenSide['bottomSide']= np.NaN   
        
    if useRows['calibFocal'].isnull().any():
        screenSide['rightSide'] = np.NaN
        screenSide['leftSide'] = np.NaN
        screenSide['topSide'] = np.NaN
        screenSide['bottomSide'] = np.NaN
 
    return RMs,useRows,screenSide

In [6]:
RM, lookDetected, ss=lookAlgorithm(df,'lookAtPoint')

In [7]:
RM

,Cali,calibFocal,rowIndex,rollingX,rollingY,props
0,0,topLeft,89,32.1,10.86,35.48
1,0,topLeft,90,31.78,10.76,33.87
2,0,topLeft,91,31.61,10.48,35.0
3,0,topLeft,92,31.4,10.57,32.79
4,0,topLeft,93,31.07,10.51,32.79
...,...,...,...,...,...,...
275,3,topRight,594,114.78,0.84,100.0
276,3,topRight,595,114.66,0.66,100.0
277,3,topRight,596,114.57,0.46,100.0
278,3,topRight,597,114.47,0.39,100.0


In [8]:
lookDetected

,index,Cali,calibFocal,rowIndex,rollingX,rollingY,props,calibrationSet
0,17,0,topLeft,106,25.33,9.66,83.33,1.0
1,100,1,bottomRight,266,113.45,14.3,85.25,1.0
2,167,2,bottomLeft,409,42.37,18.47,83.61,1.0
3,237,3,topRight,556,111.09,6.09,86.67,1.0


In [9]:
ss

{'rightSide': 113.45, 'leftSide': 25.33, 'topSide': 7.88, 'bottomSide': 16.38}